# Build a Chatbot

* Conversational RAG
* Agents

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BABB36EDE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BABB36F9B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Abhi and I am AI developer")])

AIMessage(content="Hi Abhi, it's nice to meet you! \n\nThat's awesome that you're an AI developer. What kind of AI projects are you working on? I'm always interested to hear about what people are creating in the field.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 20, 'total_tokens': 74, 'completion_time': 0.098181818, 'prompt_time': 7.7569e-05, 'queue_time': 0.020020651, 'total_time': 0.098259387}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7af5f601-e0e0-41ab-a90d-eeaa7872ce00-0', usage_metadata={'input_tokens': 20, 'output_tokens': 54, 'total_tokens': 74})

In [5]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Abhi and I am AI developer"),
    AIMessage(content="Hello Krish! It's nice to meet you .\nn\n As an AI developer, what kind of work you do?"),
    HumanMessage(content="Hey What's my name and what do I do?")
])

AIMessage(content="You said your name is Abhi and that you are an AI developer! 😊  \n\nIs there anything else you'd like to tell me about your work?  I'm curious to learn more.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 68, 'total_tokens': 112, 'completion_time': 0.08, 'prompt_time': 0.001824214, 'queue_time': 0.021721956, 'total_time': 0.081824214}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d477c9d8-93a2-47ab-81a7-5e908ae3b610-0', usage_metadata={'input_tokens': 68, 'output_tokens': 44, 'total_tokens': 112})

### Mesage History
To wrap our model and make it stateful. It keeps track of inputs and outputs of model and store them in some datastore

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response = with_message_history.invoke([HumanMessage(content="Hi , My name is Abhijeet and I am an AI developer")],
                            config=config)

In [13]:
response.content

"Hi Abhijeet, \n\nIt's great to meet you!  Being an AI developer is really exciting.  What are you working on? \n\nI'm always interested to hear about the cool projects people are creating.  Are you working with any specific AI models or technologies?\n"

In [14]:
with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config)

AIMessage(content='Your name is Abhijeet.  😊  I remember that from our first introduction! \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 264, 'total_tokens': 286, 'completion_time': 0.04, 'prompt_time': 0.00830962, 'queue_time': 0.02057022, 'total_time': 0.04830962}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f683af6-d196-4c1d-b4fb-d04858417eb0-0', usage_metadata={'input_tokens': 264, 'output_tokens': 22, 'total_tokens': 286})

In [19]:
## change the config --> session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config1)

response.content

'As an AI, I have no access to your personal information, including your name. To tell me your name, simply type it out! 😊\n'

In [20]:
response = with_message_history.invoke([HumanMessage(content="Hey my name is John")],
                            config=config1)

response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today?\n"

In [21]:
response = with_message_history.invoke([HumanMessage(content="What is my name?")],
                            config=config1)

response.content

'You told me your name is John!  😊  \n\nDid you forget?  \n'

### Prompt templates

Prompt Templates help to turn raw uuser information into a format that the LLM can work with. In this case,  the raw user input is just a message, which we are passing 

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate(
    [
        ("system",
         "You are a helpful asistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [23]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is John")]})

AIMessage(content='Hello John, nice to meet you! \n\nHow can I help you today?  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 31, 'total_tokens': 54, 'completion_time': 0.041818182, 'prompt_time': 0.00033562, 'queue_time': 0.022170178999999998, 'total_time': 0.042153802}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1441e26c-41c8-469c-a2ca-7a87931a21ca-0', usage_metadata={'input_tokens': 31, 'output_tokens': 23, 'total_tokens': 54})

In [26]:
config = {"configurable":{"session_id":"chat3"}}
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [27]:

response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config,
)

response.content

"I still don't know your name.  You haven't told me yet!  \n\nWhat can I call you? 😊 \n"

In [28]:

prompt = ChatPromptTemplate(
    [
        ("system",
         "You are a helpful asistant. Answer all questions to the best of your ability inn {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [29]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 👋 \n\nमैं आपकी मदद करने के लिए यहाँ हूँ।  आप मुझसे जो कुछ भी पूछेंगे, मैं अपनी पूरी कोशिश करूँगा। 😊 \n\nआप क्या जानना चाहते हैं? 🤔\n'

In [30]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config = {"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi my name is Krish")],"language":"Hindi"},
    config=config
)

response.content

'नमस्ते कृष्ण!  😊 \n\nमुझे बहुत खुशी है कि आप मुझसे जुड़ रहे हैं। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको मदद कर सकूँ। \n'

In [32]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="What is my name")],"language":"Hindi"},
    config=config
)

response.content

'आपका नाम कृष्ण है। 😊 \n'

### Managing the conversation history
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages wll grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
"trim_messages" helper to reduce how many messages we're sending to the model. Th trimmer allows to specify how many tokens we want to keep, along with parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="hii! I am bob"),
    AIMessage(content="Hi! Bob"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="That's great"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="2+2 is 4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No probelm"),
    HumanMessage(content="Having fun"),
    AIMessage(content="Yes!"),
]

trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\TUF GAMING\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\TUF GAMING\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hii! I am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi! Bob', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's great", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='2+2 is 4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No probelm', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like")],
    "language":"English"
    }
)

AIMessage(content='You said you like vanilla ice cream! 😊  \n\n\n\nDo you want to talk about anything else?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 111, 'total_tokens': 134, 'completion_time': 0.041818182, 'prompt_time': 0.00561043, 'queue_time': 0.020339289, 'total_time': 0.047428612}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3750861d-605c-4351-be40-2f73b8ad5bb8-0', usage_metadata={'input_tokens': 111, 'output_tokens': 23, 'total_tokens': 134})

In [37]:
## Let's wrap this thing in message history
with_message_history= RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [38]:
response=with_message_history.invoke(
    {'messages':messages + [HumanMessage(content="What is my name")],"language":"English"},
    config=config
)

response.content

'Your name is Bob. 😊  You told me earlier!\n'